In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interactive, Layout, HTML, HBox, VBox
import numpy as np
import warnings
from inspect import getargspec
from typing import Callable
import yaml

from models.interactive_parameter import FloatParameter, IntParameter, InteractiveParameter

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

**Generalised logistic function (GLF)**

![y = A + \frac{K - A}{\left( C + Q \cdot e^{-By} \right)^{ 1 / \nu} }](assets/GLF.png)

taken from https://en.wikipedia.org/wiki/Generalised_logistic_function

**Inverse GLF**
The following equation is the one used in this simulation.

![y = \frac{1}{B} \cdot ln \left( \frac{ \left( \frac{K - A}{x - A} \right)^\nu -C }{Q} \right) + E](assets/GLF_inverse.png)

A bunch of parameter can be removed, as they are not used:
- `A` is the minimum value of the x axis, which will always be `0`
- `C` and `Q` are form parameters that are meant to be set to `1` by default

With those parameters replaced by their default values, here is the simplified equation used:

![y = \frac{1}{B} \cdot ln \left( \left( \frac{K}{x} \right)^\nu -1 \right) + E](assets/GLF_inverse_simplified.png)

This equation can be generalised in other dimensions as follow:
 

In [2]:
def equation_multi(X, Y, E, B1, K1, nu1, B2, K2, nu2):
    try:
        inner1 = np.power((K1/ X), nu1) - 1
        inner2 = np.power((K2/ Y), nu2) - 1

        output: np.ndarray = (1/B1) * np.log(inner1) + (1/B2) * np.log(inner2) + E
    except ZeroDivisionError:
        output: np.ndarray = np.zeros_like(X)*np.nan

    return output

def equation_single(X, E, B, K, nu):
    try:
        inner = np.power((K/ X), nu) - 1
        output: np.ndarray = (1/B) * np.log(inner) + E
    except ZeroDivisionError:
        output: np.ndarray = np.zeros_like(X)*np.nan

    return output

def plot_static(alpha, theta, E, B1, K1, nu1, B2, K2, nu2, node_count, max_node_count, staked_tokens, max_token_supply):
    ranges = [
        np.arange(0, 1, 0.001),
        np.arange(0, 1, 0.001)
    ]
    grid = np.meshgrid(*ranges)
    T = equation_multi(*grid, E, B1, K1, nu1, B2, K2, nu2)
    T_x = equation_single(ranges[0], E, B1, K1, nu1)
    T_y = equation_single(ranges[1], E, B2, K2, nu2)

    fig = plt.figure(figsize=(15, 6))

    axes = [
        fig.add_subplot(1,4,(1,2),projection='3d'),
        fig.add_subplot(1,4,3),
        fig.add_subplot(1,4,4)
    ]

    token_ratio = staked_tokens / max_token_supply
    node_ratio = node_count / max_node_count

    # 3D plot 
    ax = axes[0]
    apr = equation_multi(token_ratio, node_ratio, E, B1, K1, nu1, B2, K2, nu2)
    ax.plot_surface(*grid, T, linewidth=0.5, cmap=cm.viridis, rstride=20, cstride=20)
    ax.contour(*grid, T, zdir='x', offset=-.5, cmap='coolwarm')
    ax.contour(*grid, T, zdir='y', offset=-.5, cmap='coolwarm')
    ax.plot([token_ratio]*2, [node_ratio]*2, [0, apr], color='k', linestyle='dashed')
    ax.plot([-.5, token_ratio], [-.5, node_ratio], [apr]*2, color='k', linestyle='dashed')

    ax.text2D(0.5, 0.95, 
            f'APR: {apr:.2f}%', 
            transform=ax.transAxes, 
            horizontalalignment='center', 
            bbox=dict(facecolor='red', alpha=0.5))
    
    ax.set_xlabel('Economic Security')
    ax.set_ylabel(f'Network size (max: {max_node_count})')
    ax.set_zlabel('APR (%)')

    ax.set_xlim(-.5, 1)
    ax.set_ylim(-.5, 1)
    ax.set_zlim(0, 15)

    ax.set_xticks(np.arange(0, 1, 0.2))
    ax.set_yticks(np.arange(0, 1, 0.2))
    ax.view_init(alpha, theta)
    ax.set_box_aspect(aspect=None, zoom=0.9)

    # 2D plot
    ax = axes[1]
    apr = equation_single(token_ratio, E, B1, K1, nu1)
    ax.plot(ranges[0], T_x, label='X', color='blue')
    ax.vlines(token_ratio, 0, apr, color='k', linestyle='dashed')
    ax.hlines(apr, -0.1, token_ratio, color='k', linestyle='dashed')

    ax.text(0.5, 0.95, 
        f'APR: {apr:.2f}%', 
        transform=ax.transAxes, 
        horizontalalignment='center', 
        bbox=dict(facecolor='red', alpha=0.5))
    
    ax.set_xlabel('Economic security')
    ax.set_ylabel('APR (%)')

    ax.set_xlim(-.1, 1.1)
    ax.set_ylim(0, 15)
    ax.grid()

    ax = axes[2]
    apr = equation_single(node_ratio, E, B2, K2, nu2)
    ax.plot(ranges[1], T_y, label='Y', color='red')
    ax.vlines(node_ratio, 0, apr, color='k', linestyle='dashed')
    ax.hlines(apr, -0.1, node_ratio, color='k', linestyle='dashed')

    ax.text(0.5, 0.95, 
            f'APR: {apr:.2f}%', 
            transform=ax.transAxes, 
            horizontalalignment='center', 
            bbox=dict(facecolor='red', alpha=0.5))
    
    ax.set_xlabel(f'Network size (max: {max_node_count})')

    ax.set_xlim(-.1, 1.1)
    ax.set_ylim(0, 15)
    ax.grid()

    plt.subplots_adjust(wspace=0.2)

def plot_interactive(callback: Callable, params: list[InteractiveParameter]):
    sliders_groups = {item.name: item.group for item in params}
    sliders_list = {item.name: item.slider() for item in params}

    args = getargspec(callback).args
    widgets = interactive(callback, **sliders_list)
    controls = widgets.children[:-1]
    graph = widgets.children[-1]
    
    grouped_widgets = {group: [] for group in set([item.group for item in params])}
    for name, widget in zip(args, controls):
        grouped_widgets[sliders_groups[name]].append(widget)

    widget_title_group = [
        VBox([
            HTML(value=f"<b>{name}</b>"), 
            HBox(widget_group, layout = Layout(flex_flow='row wrap')), 
            HTML(value="<br>")
        ])
        for name, widget_group in grouped_widgets.items()
    ]

    ordered_group = [x for _, x in sorted(zip(grouped_widgets.keys(), widget_title_group))]

    display(VBox(ordered_group))
    display(graph)

### Choice of based parameter for APR calculation

Although the APR could be a variable of many network parameters, those which makes most sense to use are the following ones.

#### Economic security ratio
This ratio is calculated as the value of token staked in the network divided by the market capitalisation.

In [3]:
with open('sigmoid.config.yaml', 'r') as file:
    config = list(yaml.safe_load_all(file))

groups_config, params_config = config
groups = {item['name']:item['description'] for item in groups_config}

params = []
params += [FloatParameter(**item) for item in params_config['floats']] 
params += [IntParameter(**item) for item in params_config['ints']]

for param in params:
    param.group = groups[param.group]

plot_interactive(plot_static, params)

Output()